In [1]:
# import packages 

import cv2
import numpy as np
import os
import datetime
import time
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# folder path and name
project_path = os.getcwd()
parent_path = os.path.abspath(os.path.join(project_path, os.pardir))
# video_folder = os.path.join(parent_path,'fisheye')
# image_folder = os.path.join(parent_path,'snapshot_archive')
video_folder = 'E:\\video archive\\fisheye'
image_folder = 'E:\\snapshot_archive'
image_name_format = '%Y%m%d%H%M%S'

# Operating parameter
capture_interval = datetime.timedelta(minutes=1)  # in s
start_date = datetime.datetime(2018,1,4,12)

In [2]:
def crop_central_image(img, x_length, y_length):
    image_shape = np.array(img.shape[0:2])
    cropped_shape = np.array([x_length, y_length])
    x_low, y_low = ((image_shape - cropped_shape) / 2).astype('int')
    x_high, y_high = ((image_shape + cropped_shape) / 2).astype('int')
    img = img[x_low:x_high, y_low:y_high, :]
    return img

In [3]:
def listdir_noini(path):
    f_list = []
    for f in os.listdir(path):
        _, file_extension = os.path.splitext(f)
        if not file_extension == '.ini':
            f_list.append(f)
            
    return sorted(f_list)

In [4]:
def parse_video_name(video_name,start_date):
    # ignore the file if it's an image file
    _,extension = os.path.splitext(video_name)
    if extension == '.jpg':
        return None, False
    
    # Interpret video file name into time stamp
    if len(video_name) < 30:
        video_name_format = "%Y%m%d-%H%M%S-01.avi"
    else:
        video_name = video_name.split('_')[-3]
        video_name_format = "%Y%m%d%H%M%S"
    
    # obtaining initial time
    try:
        initial_time = datetime.datetime.strptime(video_name, video_name_format)
    except:# there are some other files in the folder
        print("can't handle ", video_name)
        return None, False

    # Only execute the frame query starting from start date
    if initial_time< start_date:
        #print('Skipped files that are not in time range!')
        return None, False

    
    return initial_time, True

In [5]:
# find start date of the current capture process
# This should be right after the last date of the last capturing process
# previous_all_times = np.load(os.path.join(image_folder,'all_times.npy'))
# start_date = previous_all_times[-1]

# print('Last time, snapshot is finished for: ',start_date)

In [6]:
# Snapshot from video at one minute interval, and formulate input output matrix from it

# Initialization
number_images = int((datetime.datetime.now()-start_date).total_seconds()/60)
all_times = np.zeros(number_images).astype(datetime.datetime)

# time and frame counter
tic = time.process_time()
frame_cnt = 0

# Step through every single video, but only open the streams that are relevant
# list all years  
years_list = listdir_noini(os.path.join(video_folder))
for year in years_list: # cycle through the years
    months_list = listdir_noini(os.path.join(video_folder,year))
    for month in months_list: # cycle through the months
        dates_list  = listdir_noini(os.path.join(video_folder,year,month))
        for date in dates_list: # cycle through each day
            curr_folder = os.path.join(video_folder,year,month,date)
            video_filenames = listdir_noini(curr_folder)
            print('processing ',month,'-',date)
            
            for video_name in video_filenames: # cycle through each file of the day
                # Attempt to parse the filename
                initial_time,is_relevant = parse_video_name(video_name,start_date)
                # if the file is deemed irrelevant, move on to the next
                if is_relevant == False:
                    continue
                
                print('Capturing images from video stream:',video_name)
                # shift to the first exact minute
                shift = datetime.timedelta(seconds=60 - initial_time.second)  # in s
                video_pos_time = (60 - initial_time.second) * 1000 # in ms
                initial_time += shift
                curr_time = initial_time
    
                # open video stream
                video_path = os.path.join(curr_folder,video_name)
                cap = cv2.VideoCapture(video_path)

                while cap.isOpened():
                    cap.set(0, video_pos_time)
                    ret, frame = cap.read()
                    frame_pos_frac =  cap.get(cv2.CAP_PROP_POS_FRAMES)/cap.get(cv2.CAP_PROP_FRAME_COUNT)
                    
                    if ret and frame_pos_frac <= 0.999:
                        # crop and save the central square
                        frame = crop_central_image(frame,frame.shape[0],frame.shape[0])
                        cv2.imwrite(os.path.join(image_folder, curr_time.strftime(image_name_format) + ".jpg"), frame)

                        # resizing the image to output_img_shape
                        all_times[frame_cnt] = curr_time
                        frame_cnt+=1

                        #Move forward in time    
                        video_pos_time += capture_interval.total_seconds() * 1000  # in ms
                        curr_time += capture_interval
                        if curr_time.minute%10 == 0 and curr_time.second == 0:
                            print('processing ',curr_time.strftime('%H%M%S'))
                    else :
                        break
                cap.release()
            print('time_elapsed',time.process_time()-tic)


processing  12 - 14
can't handle  20161214.db
time_elapsed 0.0
processing  12 - 20
can't handle  20161220.db
time_elapsed 0.046875
processing  12 - 21
can't handle  20161221.db
time_elapsed 0.046875
processing  12 - 22
can't handle  20161222.db
time_elapsed 0.0625
processing  12 - 24
can't handle  20161224.db
time_elapsed 0.0625
processing  12 - 25
can't handle  20161225.db
time_elapsed 0.0625
processing  12 - 26
can't handle  20161226-112856-01 (1).avi
can't handle  20161226.db
time_elapsed 0.0625
processing  12 - 30
can't handle  20161230.db
time_elapsed 0.078125
processing  01 - 01
can't handle  20170101.db
time_elapsed 0.078125
processing  01 - 02
can't handle  20170102.db
time_elapsed 0.078125
processing  01 - 03
can't handle  20170103-184819-01.___
can't handle  20170103.db
time_elapsed 0.109375
processing  01 - 06
can't handle  20170106.db
time_elapsed 0.109375
processing  01 - 07
can't handle  20170107.db
time_elapsed 0.125
processing  01 - 08
can't handle  20170108-220704-01._

processing  124000
processing  125000
processing  130000
processing  131000
Capturing images from video stream: roof_top_Camera1_20180104130915_20180104134413_3836838.mp4
processing  132000
processing  133000
processing  134000
Capturing images from video stream: roof_top_Camera1_20180104134414_20180104142029_3883751.mp4
processing  135000
processing  140000
processing  141000
processing  142000
Capturing images from video stream: roof_top_Camera1_20180104142029_20180104145439_3930382.mp4
processing  143000
processing  144000
processing  145000
Capturing images from video stream: roof_top_Camera1_20180104145439_20180104153331_3976597.mp4
processing  150000
processing  151000
processing  152000
processing  153000
Capturing images from video stream: roof_top_Camera1_20180104153331_20180104161026_4023615.mp4
processing  154000
processing  155000
processing  160000
processing  161000
Capturing images from video stream: roof_top_Camera1_20180104161026_20180104172526_4070164.mp4
processing  

processing  064000
Capturing images from video stream: roof_top_Camera1_20180106064710_20180106065644_6487803.mp4
processing  065000
Capturing images from video stream: roof_top_Camera1_20180106065644_20180106074014_6532590.mp4
processing  070000
processing  071000
processing  072000
processing  073000
processing  074000
Capturing images from video stream: roof_top_Camera1_20180106074014_20180106082401_6575992.mp4
processing  075000
processing  080000
processing  081000
processing  082000
Capturing images from video stream: roof_top_Camera1_20180106082401_20180106095519_6620047.mp4
processing  083000
processing  084000
processing  085000
processing  090000
processing  091000
processing  092000
processing  093000
processing  094000
processing  095000
Capturing images from video stream: roof_top_Camera1_20180106095520_20180106110315_6664396.mp4
processing  100000
processing  101000
processing  102000
processing  103000
processing  104000
processing  105000
processing  110000
Capturing im

Capturing images from video stream: roof_top_Camera1_20180107193820_20180107194648_8744193.mp4
processing  194000
Capturing images from video stream: roof_top_Camera1_20180107194648_20180107195517_8786858.mp4
processing  195000
Capturing images from video stream: roof_top_Camera1_20180107195517_20180107200000_8829380.mp4
processing  200000
time_elapsed 147498.15625
processing  01 - 20180115
Capturing images from video stream: roof_top_Camera1_20180115083119_20180115095811_715025.mp4
processing  084000
processing  085000
processing  090000
processing  091000
processing  092000
processing  093000
processing  094000
processing  095000
Capturing images from video stream: roof_top_Camera1_20180115095811_20180115105810_758984.mp4
processing  100000
processing  101000
processing  102000
processing  103000
processing  104000
processing  105000
Capturing images from video stream: roof_top_Camera1_20180115105810_20180115125847_805454.mp4
processing  110000
processing  111000
processing  112000
p

processing  154000
processing  155000
processing  160000
processing  161000
processing  162000
processing  163000
processing  164000
processing  165000
processing  170000
processing  171000
Capturing images from video stream: roof_top_Camera1_20180117171026_20180117175926_2429769.mp4
processing  172000
processing  173000
processing  174000
processing  175000
processing  180000
Capturing images from video stream: roof_top_Camera1_20180117175926_20180117181716_2474323.mp4
processing  181000
Capturing images from video stream: roof_top_Camera1_20180117181716_20180117183513_2519769.mp4
processing  182000
processing  183000
Capturing images from video stream: roof_top_Camera1_20180117183513_20180117185209_2565106.mp4
processing  184000
processing  185000
Capturing images from video stream: roof_top_Camera1_20180117185209_20180117190904_2609704.mp4
processing  190000
processing  191000
Capturing images from video stream: roof_top_Camera1_20180117190905_20180117192618_2655657.mp4
processing  

...

processing  155000
Capturing images from video stream: roof_top_Camera1_20180124155732_20180124174441_8335981.mp4
processing  160000
processing  161000
processing  162000
processing  163000
processing  164000
processing  165000
processing  170000
processing  171000
processing  172000
processing  173000
processing  174000
Capturing images from video stream: roof_top_Camera1_20180124174441_20180124180329_8380876.mp4
processing  175000
processing  180000
Capturing images from video stream: roof_top_Camera1_20180124180330_20180124182046_8427054.mp4
processing  181000
processing  182000
Capturing images from video stream: roof_top_Camera1_20180124182046_20180124183735_8471997.mp4
processing  183000
Capturing images from video stream: roof_top_Camera1_20180124183735_20180124185411_8518252.mp4
processing  184000
processing  185000
Capturing images from video stream: roof_top_Camera1_20180124185411_20180124191110_8563539.mp4
processing  190000
processing  191000
Capturing images from video str

processing  200000
Capturing images from video stream: roof_top_Camera1_20180125195445_20180125200000_9703116.mp4
processing  200000
time_elapsed 414079.578125
processing  01 - 20180126
Capturing images from video stream: roof_top_Camera1_20180126055956_20180126061319_952808.mp4
processing  061000
Capturing images from video stream: roof_top_Camera1_20180126061319_20180126063149_983057.mp4
processing  062000
processing  063000
Capturing images from video stream: roof_top_Camera1_20180126063149_20180126065016_1028000.mp4
processing  064000
processing  065000
Capturing images from video stream: roof_top_Camera1_20180126065016_20180126083101_1072202.mp4
processing  070000
processing  071000
processing  072000
processing  073000
processing  074000
processing  075000
processing  080000
processing  081000
processing  082000
processing  083000
Capturing images from video stream: roof_top_Camera1_20180126083101_20180126101500_1115602.mp4
processing  084000
processing  085000
processing  090000

processing  092000
Capturing images from video stream: roof_top_Camera1_20180128092901_20180128110821_2876456.mp4
processing  094000
processing  095000
processing  100000
processing  101000
processing  102000
processing  103000
processing  104000
processing  105000
processing  110000
Capturing images from video stream: roof_top_Camera1_20180128110821_20180128124007_2920043.mp4
processing  111000
processing  112000
processing  113000
processing  114000
processing  115000
processing  120000
processing  121000
processing  122000
processing  123000
processing  124000
Capturing images from video stream: roof_top_Camera1_20180128124007_20180128141017_2963298.mp4
processing  125000
processing  130000
processing  131000
processing  132000
processing  133000
processing  134000
processing  135000
processing  140000
processing  141000
Capturing images from video stream: roof_top_Camera1_20180128141017_20180128153746_3006444.mp4
processing  142000
processing  143000
processing  144000
processing  

processing  173000
processing  174000
processing  175000
processing  180000
Capturing images from video stream: roof_top_Camera1_20180130180458_20180130182303_4669841.mp4
processing  181000
processing  182000
Capturing images from video stream: roof_top_Camera1_20180130182303_20180130183948_4713311.mp4
processing  183000
processing  184000
Capturing images from video stream: roof_top_Camera1_20180130183949_20180130185625_4757551.mp4
processing  185000
Capturing images from video stream: roof_top_Camera1_20180130185625_20180130191300_4800968.mp4
processing  190000
processing  191000
Capturing images from video stream: roof_top_Camera1_20180130191300_20180130192935_4845361.mp4
processing  192000
processing  193000
Capturing images from video stream: roof_top_Camera1_20180130192935_20180130194610_4889235.mp4
processing  194000
Capturing images from video stream: roof_top_Camera1_20180130194610_20180130200000_4934781.mp4
processing  195000
processing  200000
time_elapsed 536179.046875
proc

KeyboardInterrupt: 

In [7]:
# Cropping relevant information
all_times = np.concatenate((previous_all_times, all_times[:frame_cnt]))
        
#storing information
np.save(os.path.join(image_folder,'all_times.npy'),all_times)


In [10]:
all_times

array([datetime.datetime(2017, 3, 9, 6, 0),
       datetime.datetime(2017, 3, 9, 6, 1),
       datetime.datetime(2017, 3, 9, 6, 2), ...,
       datetime.datetime(2018, 1, 4, 16, 56),
       datetime.datetime(2018, 1, 4, 16, 57),
       datetime.datetime(2018, 1, 4, 16, 58)], dtype=object)